In [ ]:
pip install apscheduler cloudscraper pandas beautifulsoup4

In [ ]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time

# Create a CloudScraper session
scraper = cloudscraper.create_scraper()

# Base URL (modify `page` parameter)
base_url = "https://www.bbb.org/search?city=new-york&find_country=USA&find_entity=60729-000&find_id=4669_18000&find_latlng=40.762801%2C-73.977818&find_loc=New%20York%2C%20NY&find_text=Real%20Estates&find_type=Category&page=1&sort=Distance&state=NY"

# User-Agent headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

# Pagination settings
max_pages = 20  # Adjust this if you need more pages
all_data = []

for page in range(1, max_pages + 1):
    print(f"📄 Scraping page {page}...")

    # Get page content
    url = base_url.format(page)
    response = scraper.get(url, headers=headers)

    if response.status_code != 200:
        print(f"❌ Failed to fetch page {page}. Status Code: {response.status_code}")
        break

    soup = BeautifulSoup(response.text, "html.parser")
    main_div = soup.find("div", class_="stack stack-space-20")

    if not main_div:
        print("⚠️ No more results found. Stopping.")
        break

    # Find all contractor listings
    contractors = main_div.find_all("div", class_="card result-card")

    if not contractors:
        print("⚠️ No more contractors found. Ending pagination.")
        break

    for contractor in contractors:
        try:
            name = contractor.find("h3").find("a", class_="text-blue-medium").text.strip()
        except AttributeError:
            name = "N/A"

        try:
            company = contractor.find("span").text.strip()
        except AttributeError:
            company = "N/A"

        try:
            industry = contractor.find("p", class_="bds-body text-size-4 text-gray-70").text.strip()
        except AttributeError:
            industry = "N/A"

        try:
            contact_tag = contractor.find("a", class_="text-black")
            contact = contact_tag["href"].replace("tel:", "").strip() if contact_tag and "href" in contact_tag.attrs else "N/A"
        except AttributeError:
            contact = "N/A"

        try:
            location = contractor.find("p", class_="bds-body text-size-5 text-gray-70").text.strip()
        except AttributeError:
            location = "N/A"

        all_data.append({
            "Name": name,
            "Company": company,
            "Industry": industry,
            "Contact": contact,
            "Location": location
        })

    # Avoid getting blocked by the server
    time.sleep(2)  # Adjust delay if needed

# Save data to DataFrame
df = pd.DataFrame(all_data)
print(df.head())  # Display first few rows

# Save to CSV
df.to_csv("bbb_contractors_all_pages.csv", index=False)
print(f"✅ Data from {len(df)} contractors saved to bbb_contractors_all_pages.csv")


📄 Scraping page 1...
📄 Scraping page 2...
📄 Scraping page 3...
📄 Scraping page 4...
📄 Scraping page 5...
📄 Scraping page 6...
📄 Scraping page 7...
📄 Scraping page 8...
📄 Scraping page 9...
📄 Scraping page 10...
📄 Scraping page 11...
📄 Scraping page 12...
📄 Scraping page 13...
📄 Scraping page 14...
📄 Scraping page 15...
📄 Scraping page 16...
📄 Scraping page 17...
📄 Scraping page 18...
📄 Scraping page 19...
📄 Scraping page 20...
                          Name                      Company  \
0            NYC Office Suites            NYC Office Suites   
1   Newcastle Investment Corp.   Newcastle Investment Corp.   
2  Windsor Development Company  Windsor Development Company   
3          Lefrak Organization          Lefrak Organization   
4  The Jack Parker Corporation  The Jack Parker Corporation   

                         Industry          Contact  \
0  Real Estates, Leasing Services  +1-646-380-2442   
1     Trust Company, Real Estates              N/A   
2                    Real Es